In [1]:
from config import Config
import numpy as np
from shapely.geometry import LineString, Point, MultiPoint, MultiLineString
from shapely.affinity import scale

# from rasterstats import zonal_stats
# import rasterio as rio
import geopandas as gpd
# from rasterio.mask import mask
import os
# import rasterstats as rasstats

out_crs = 'EPSG:28356'
import pandas as pd

cfg = Config()

In [2]:
cfg_test = cfg.pine_dirs
cfg_test

{'folder': 'E:\\Brisbane_Pine GIS QGIS_2018\\Brisbane GIS data\\Lidar_Pine\\Pine shapefiles',
 'lidar': 'pine_lidar_clip.asc',
 'thalwegs': 'NPR_thalweg.geojson',
 'banks': 'NPR_banks.geojson',
 'points': 'NPR_data_points.geojson'}

In [3]:
line = LineString([(0,0), (3,3)])
line2 = LineString([(0,1), (1,0)])
line3 = LineString([(0,3), (1,3)])

pointr = Point(2,1)

In [4]:
line.intersection(line3).wkt

'GEOMETRYCOLLECTION EMPTY'

In [5]:
lspot = line.project(pointr)

lpoint = line.interpolate(lspot)
lpoint2 = line.interpolate(lspot-0.3)

In [6]:
scale(line, 2, 2).wkt

'LINESTRING (-1.5 -1.5, 4.5 4.5)'

In [7]:
lpoint = line.interpolate(100)

In [8]:
lpoint.wkt

'POINT (3 3)'

In [9]:
line.project(lpoint, normalized=True), lspot-0.3

(1.0, 1.8213203435596423)

In [10]:
def find_dir(p1, p2):
    p1 = np.array(p1.xy)
    p2 = np.array(p2.xy)
    dx = p2[0][0] - p1[0][0]
    dy = p2[1][0] - p1[1][0]   
    return np.arctan2(dy,dx)


def find_dist(p1, p2):
    p1 = np.array(p1.xy)
    p2 = np.array(p2.xy)
    dx = p2[0][0] - p1[0][0]
    dy = p2[1][0] - p1[1][0]  
    
    dist = np.sqrt(dx**2+dy**2)
    return dist
    
def lin_ref(point, line):
    lspot = line.project(point)

    lpoint = line.interpolate(lspot)
    ldir = np.degrees(find_dir(lpoint, point))
    ldist = find_dist(lpoint, point)

    
    return lpoint, ldir, ldist
    

In [11]:
thalwegs.head()

NameError: name 'thalwegs' is not defined

In [12]:
type(thalwegs[thalwegs['Reach'] == 'LC1'].geometry)

NameError: name 'thalwegs' is not defined

In [13]:
# thalwegs.loc[0]
# thalwegs[thalwegs['Reach'] == 'UBR1'].geometry.item()
thalwegs.head()

NameError: name 'thalwegs' is not defined

In [14]:
# for reach in list(thalwegs['Reach'].unique()):
#     # print(weg)
#     print(reach, type(thalwegs[thalwegs['Reach']==reach].geometry.iloc[0]), thalwegs[thalwegs['Reach']==reach].geometry.shape)
#     LineString(thalwegs[thalwegs['Reach']==reach].geometry.iloc[0][0])
    

In [15]:
# ras = src.read(1)
thalwegs[thalwegs['Reach']==reach].geometry.iloc[0]
    

NameError: name 'thalwegs' is not defined

In [16]:
# def raster_stats(raster, point, buffer = 2.0):
#     pt = point.buffer(buffer)
    
#     mask_im, con = mask(raster, [pt], crop=True)
# #     print(mask_im)
#     b_mean = np.mean(mask_im)
#     b_stdev = np.std(mask_im)
#     return b_mean, b_stdev

In [17]:


# middle = Point((484054.0 -5000), (6996682.0-5000))

In [18]:
# bmean, bstdev = raster_stats(src, middle)
# src.bounds

In [19]:
# bmean, bstdev, mid
# LineString([Point((1,1)), Point((3,3))])

In [20]:
def get_geom_data(pname, point,reach, thalwegs, banklines):
    
    thalweg = thalwegs[thalwegs['Reach'] == reach].geometry
    #0.6.1 geometry array change, somtiems these are multilines and come off as geoseries
    if type(thalweg) ==gpd.geoseries.GeoSeries:
        thalweg = thalweg.iloc[0]
    
    banks = banklines[banklines['Reach']==reach]

    rbank = banks[banks['Bank'] == 'R'].geometry
    if type(rbank) ==gpd.geoseries.GeoSeries:
        rbank = rbank.iloc[0]
    if type(rbank) == MultiLineString:
        rbank = LineString(rbank[0])

    lbank = banks[banks['Bank'] == 'L'].geometry
    if type(lbank) ==gpd.geoseries.GeoSeries:
        lbank = lbank.iloc[0]
    if type(lbank) == MultiLineString:
        lbank = LineString(lbank[0])
    
    tp, tdir, tdist = lin_ref(point, thalweg)
    rp, rdir, rdist = lin_ref(point, rbank)
    lp, ldir, ldist = lin_ref(point, lbank)
    bp, bbfdir, bfwidth = lin_ref(rp, lbank) # grabbing channel width here because why not normalize things ;)
    # new analysis, going to project a line between sample and thalweg, and get bank points from that
   

    point = Point(xy[:2] for xy in list(point.coords))
    print(point, tp)
    tline = LineString([point, tp])
    tscale = 1.25*bfwidth/tdist
    tline = scale(tline, tscale, tscale)
    rp = rbank.intersection(tline)
    if rp.wkt == 'GEOMETRYCOLLECTION EMPTY':
        
        print('line too small scalling up to rb')
        ct=0
        while rp.wkt == 'GEOMETRYCOLLECTION EMPTY'and (ct<10):
            tline = scale(tline, 1.2, 1.2)
            rp = rbank.intersection(tline)
            ct+=1
    lp = lbank.intersection(tline)
    if lp.wkt == 'GEOMETRYCOLLECTION EMPTY':
        print('line too small scalling up to lb')
        ct = 0
        while (lp.wkt == 'GEOMETRYCOLLECTION EMPTY') and (ct<10):
            tline = scale(tline, 1.2, 1.2)
            lp = lbank.intersection(tline)
            ct+=1
    if lp.wkt == 'GEOMETRYCOLLECTION EMPTY':
        lp, ldir, ldist = lin_ref(point, lbank)
    else:

        ldir = np.degrees(find_dir(lp, point))
        ldist = find_dist(lp, point)
        
    if rp.wkt == 'GEOMETRYCOLLECTION EMPTY':
        rp, rdir, rdist = lin_ref(point, rbank)
    else:
        rdir = np.degrees(find_dir(rp, point))
        rdist = find_dist(rp, point)
    
    bp, bbfdir, bfwidth = lin_ref(rp, lbank)

#     pte, ptestd = raster_stats(raster,point)
# #     pte, ptestd = raster_stats(point, raster)
#     tpte, tptestd = raster_stats(raster,tp)

#     rpte, rptestd = raster_stats(raster,rp)
#     lpte, lptestd = raster_stats(raster, lp)
    
    return pname, tp, tdir,tdist, rp, rdir,rdist, lp, ldir,ldist, bfwidth, reach, tline
    
    
    



    

    

In [21]:

# src = rio.open(cfg_test['lidar'])

In [22]:
gpd.__version__


'0.6.1'

In [23]:


cfg_test=cfg.pine_dirs
os.chdir(cfg_test['folder'])
points = gpd.read_file(cfg_test['points'])

points.shape

(127, 15)

In [24]:

?gpd.GeoDataFrame

Init signature: gpd.GeoDataFrame(*args, **kwargs)
Docstring:     
A GeoDataFrame object is a pandas.DataFrame that has a column
with geometry. In addition to the standard DataFrame constructor arguments,
GeoDataFrame also accepts the following keyword arguments:

Parameters
----------
crs : str (optional)
    Coordinate system
geometry : str or array (optional)
    If str, column to use as geometry. If array, will be set as 'geometry'
    column on GeoDataFrame.
File:           c:\users\garberj\.conda\envs\veg_an\lib\site-packages\geopandas\geodataframe.py
Type:           type
Subclasses:     


In [25]:
# stats = zonal_stats(tcirc,cfg_test['lidar'] )

In [26]:
for cfg_test, cat_name in zip([cfg.pine_dirs, cfg.bris_dirs, cfg.kobble_dirs, cfg.mary_dirs], ['NPR', 'Bris','kobble', 'Mary']):
    os.chdir(cfg_test['folder'])
    thalwegs = gpd.read_file(cfg_test['thalwegs'])
    banklines = gpd.read_file(cfg_test['banks'])
    points = gpd.read_file(cfg_test['points'])
    # because im an idiot with field names had to throw in these fucking fixes
    if 'Bank' not in list(banklines.columns):
        banklines = banklines.rename(columns={'Banks':'Bank'})
    if 'join_Reach' not in list(points.columns):
        if 'Reach' in list(points.columns):
            points = points.rename(columns = {'Reach':'join_Reach'})
        elif 'join_reach' in list(points.columns):
            points = points.rename(columns = {'join_reach':'join_Reach'})
    if 'Reach' not in list(thalwegs.columns):
        thalwegs = thalwegs.rename(columns={'reach':'Reach'})
        banklines =banklines.rename(columns={'reach':'Reach', 'bank':'Bank'})
    
    if cat_name =='Mary':
        points = points[points['join_reach'] != 'MR1B']
        
    run_an = lambda x: get_geom_data(x['name'],x['geometry'], x['join_Reach'], thalwegs=thalwegs, banklines=banklines)
    lister = points.apply(run_an, axis=1)
    name = [l[0] for l in lister]
    tp = [l[1] for l in lister]

    td = [l[2] for l in lister]
    tdis = [l[3] for l in lister]

    rp = [l[4] for l in lister]
    rd = [l[5] for l in lister]
    rdis= [l[6] for l in lister]
    lp = [l[7] for l in lister]
    ld = [l[8] for l in lister]
    ldis = [l[9] for l in lister]
    bfw = [l[10] for l in lister]
    reacher = [l[11] for l in lister]
    xs_line = [l[12] for l in lister]



    geom_dat = pd.DataFrame({'name':name,'thal_dist':tdis, 'thal_dir':td,
                             'rb_dis':rdis,'rb_dir':rd,'lb_dis':ldis, 
                             'lb_dir':ld,'bfw': bfw, 'Reach':reacher })

    rbpts = gpd.GeoDataFrame({'name':name, 'geometry':rp,'Reach':reacher}, geometry= 'geometry', crs=out_crs)
    lbpts = gpd.GeoDataFrame({'name':name, 'geometry':lp,'Reach':reacher}, geometry= 'geometry', crs=out_crs)
    tpts = gpd.GeoDataFrame({'name':name, 'geometry':tp,'Reach':reacher}, geometry= 'geometry', crs=out_crs)
    xsline = gpd.GeoDataFrame({'name':name, 'geometry':xs_line, 'Reach':reacher}, geometry='geometry', crs=out_crs)

    rbpts['type'] = 'RB'
    lbpts['type'] = 'LB'
    tpts['type'] = 'thal'

    pts = points[['name', 'geometry', 'join_Reach']].rename(columns = {	'join_Reach':'Reach'})

    pts['type'] = 'sl'

    merged_pts = pd.concat([pts, rbpts, tpts, lbpts])

    merged_buf = merged_pts.copy()

    merged_buf['geometry'] = merged_buf['geometry'].buffer(2.0)
    
    os.chdir(r'E:\Brisbane_Pine GIS QGIS_2018\GIS_scripts_dendro')

    geom_dat.to_csv(cat_name+'_geom_data_LR.csv')
    
    merged_pts.to_file(cat_name+'_merged_pts_V2.geojson', driver='GeoJSON')
    merged_buf.to_file(cat_name+'_merged_buf_V2.geojson', driver='GeoJSON')
    xsline.to_file(cat_name+'projected_XS_V2.geojson', driver='GeoJSON')

    

    

029.9326740791 6990515.861286562)
POINT (481015.9476290083 6990508.660123247) POINT (481017.7340491715 6990505.549288135)
POINT (483237.4092326298 6989727.657384681) POINT (483274.5416912902 6989671.958696692)
line too small scalling up to rb
POINT (483217.1704775327 6989755.320791477) POINT (483199.9522222055 6989669.27284944)
line too small scalling up to rb
POINT (483391.523871986 6989705.268993303) POINT (483383.0537065335 6989657.16484611)
line too small scalling up to rb
POINT (483225.1389715098 6989647.780616952) POINT (483230.889248086 6989657.090588552)
line too small scalling up to lb
POINT (483221.2796804267 6989645.781662293) POINT (483228.9893815455 6989658.264035532)
line too small scalling up to lb
POINT (483229.8975772418 6989643.799579347) POINT (483236.1136745581 6989653.863736907)
line too small scalling up to lb
POINT (483334.7619505189 6989793.914091243) POINT (483299.7392451121 6989680.128341471)
line too small scalling up to rb
POINT (483718.2894028634 6989645.11

KeyError: 'join_reach'

In [27]:
cat_name

'Mary'

In [28]:
# note just copied and pasted this fucker to get it to work and not repeat everything
if cat_name =='Mary':
        points = points[points['join_Reach'] != 'MR1B']
        
run_an = lambda x: get_geom_data(x['name'],x['geometry'], x['join_Reach'], thalwegs=thalwegs, banklines=banklines)
lister = points.apply(run_an, axis=1)
name = [l[0] for l in lister]
tp = [l[1] for l in lister]

td = [l[2] for l in lister]
tdis = [l[3] for l in lister]

rp = [l[4] for l in lister]
rd = [l[5] for l in lister]
rdis= [l[6] for l in lister]
lp = [l[7] for l in lister]
ld = [l[8] for l in lister]
ldis = [l[9] for l in lister]
bfw = [l[10] for l in lister]
reacher = [l[11] for l in lister]
xs_line = [l[12] for l in lister]



geom_dat = pd.DataFrame({'name':name,'thal_dist':tdis, 'thal_dir':td,
                         'rb_dis':rdis,'rb_dir':rd,'lb_dis':ldis, 
                         'lb_dir':ld,'bfw': bfw, 'Reach':reacher })

rbpts = gpd.GeoDataFrame({'name':name, 'geometry':rp,'Reach':reacher}, geometry= 'geometry', crs=out_crs)
lbpts = gpd.GeoDataFrame({'name':name, 'geometry':lp,'Reach':reacher}, geometry= 'geometry', crs=out_crs)
tpts = gpd.GeoDataFrame({'name':name, 'geometry':tp,'Reach':reacher}, geometry= 'geometry', crs=out_crs)
xsline = gpd.GeoDataFrame({'name':name, 'geometry':xs_line, 'Reach':reacher}, geometry='geometry', crs=out_crs)

rbpts['type'] = 'RB'
lbpts['type'] = 'LB'
tpts['type'] = 'thal'

pts = points[['name', 'geometry', 'join_Reach']].rename(columns = {	'join_Reach':'Reach'})

pts['type'] = 'sl'

merged_pts = pd.concat([pts, rbpts, tpts, lbpts])

merged_buf = merged_pts.copy()

merged_buf['geometry'] = merged_buf['geometry'].buffer(2.0)

os.chdir(r'e:\Brisbane_Pine GIS QGIS_2018\GIS_scripts_dendro')

geom_dat.to_csv(cat_name+'_geom_data_LR.csv')

merged_pts.to_file(cat_name+'_merged_pts_V2.geojson', driver='GeoJSON')
merged_buf.to_file(cat_name+'_merged_buf-V2.geojson', driver='GeoJSON')
xsline.to_file(cat_name+'projected_XS_V2.geojson', driver='GeoJSON')

POINT (471472.6644190443 7043715.712826243) POINT (471517.8726959409 7043697.366668024)
line too small scalling up to rb
POINT (471477.1353599611 7043717.716506008) POINT (471519.2030289251 7043700.644850912)
line too small scalling up to rb
POINT (471471.6942686413 7043749.048294124) POINT (471529.3506781173 7043725.650507631)
line too small scalling up to rb
POINT (471501.5945649647 7043720.651210334) POINT (471523.6840783922 7043711.686973338)
line too small scalling up to rb
POINT (471507.7706766902 7043716.234848088) POINT (471523.0185692742 7043710.047037601)
line too small scalling up to rb
POINT (471486.2636925131 7043771.453866482) POINT (471536.4202177576 7043743.028011977)
line too small scalling up to rb
POINT (471489.6426503702 7043772.569033422) POINT (471537.7200429448 7043745.321512593)
line too small scalling up to rb
POINT (471498.0895941466 7043819.770225363) POINT (471560.021082254 7043784.670994273)
line too small scalling up to rb
POINT (471483.6893444241 7043766.

### A one off for redoing NPR after resaving it and buildign it up


In [ ]:
for cfg_test, cat_name in zip([cfg.pine_dirs], ['NPR']):
    os.chdir(cfg_test['folder'])
    thalwegs = gpd.read_file(cfg_test['thalwegs'])
    banklines = gpd.read_file(cfg_test['banks'])
    points = gpd.read_file(cfg_test['points'])
    # because im an idiot with field names had to throw in these fucking fixes
    if 'Bank' not in list(banklines.columns):
        banklines = banklines.rename(columns={'Banks':'Bank'})
    if 'join_Reach' not in list(points.columns):
        if 'Reach' in list(points.columns):
            points = points.rename(columns = {'Reach':'join_Reach'})
        elif 'join_reach' in list(points.columns):
            points = points.rename(columns = {'join_reach':'join_Reach'})
    if 'Reach' not in list(thalwegs.columns):
        thalwegs = thalwegs.rename(columns={'reach':'Reach'})
        banklines =banklines.rename(columns={'reach':'Reach', 'bank':'Bank'})
    
    if cat_name =='Mary':
        points = points[points['join_reach'] != 'MR1B']
        
    run_an = lambda x: get_geom_data(x['name'],x['geometry'], x['join_Reach'], thalwegs=thalwegs, banklines=banklines)
    lister = points.apply(run_an, axis=1)
    name = [l[0] for l in lister]
    tp = [l[1] for l in lister]

    td = [l[2] for l in lister]
    tdis = [l[3] for l in lister]

    rp = [l[4] for l in lister]
    rd = [l[5] for l in lister]
    rdis= [l[6] for l in lister]
    lp = [l[7] for l in lister]
    ld = [l[8] for l in lister]
    ldis = [l[9] for l in lister]
    bfw = [l[10] for l in lister]
    reacher = [l[11] for l in lister]


    geom_dat = pd.DataFrame({'name':name,'thal_dist':tdis, 'thal_dir':td,
                             'rb_dis':rdis,'rb_dir':rd,'lb_dis':ldis, 
                             'lb_dir':ld,'bfw': bfw, 'Reach':reacher })

    rbpts = gpd.GeoDataFrame({'name':name, 'geometry':rp,'Reach':reacher}, geometry= 'geometry')
    lbpts = gpd.GeoDataFrame({'name':name, 'geometry':lp,'Reach':reacher}, geometry= 'geometry')
    tpts = gpd.GeoDataFrame({'name':name, 'geometry':tp,'Reach':reacher}, geometry= 'geometry')

    rbpts['type'] = 'RB'
    lbpts['type'] = 'LB'
    tpts['type'] = 'thal'

    pts = points[['name', 'geometry', 'join_Reach']].rename(columns = {	'join_Reach':'Reach'})

    pts['type'] = 'sl'

    merged_pts = pd.concat([pts, rbpts, tpts, lbpts])

    merged_buf = merged_pts.copy()

    merged_buf['geometry'] = merged_buf['geometry'].buffer(2.0)
    
    os.chdir(r'E:\Brisbane_Pine GIS QGIS_2018\GIS_scripts_dendro')

    geom_dat.to_csv(cat_name+'_geom_data_LR.csv')
    
    merged_pts.to_file(cat_name+'_merged_pts.geojson', driver='GeoJSON')
    merged_buf.to_file(cat_name+'_merged_buf.geojson', driver='GeoJSON')


# Part two, grabbing the end points of the thalweg for a gradient calculation

In [11]:
def line_ends(line):
    pts = list(line.coords)
    top = pts[0]
    bot = pts[-1]
    return [Point(top),Point(bot)]

In [12]:
test_ls = LineString([(1,1), (2,2), (2,1)])
test_ls
list(test_ls.coords)[0]

test_ls.length

2.414213562373095

In [13]:
cfg_test = cfg.pine_dirs
os.chdir(cfg_test['folder'])
thalwegs = gpd.read_file(cfg_test['thalwegs'])

In [14]:
thalwegs.head()

,id,Reach,geometry
0,None,LC1,(LINESTRING (473813.9128900793 6989466.0442244...
1,None,LC2,(LINESTRING (474458.6033456232 6990079.7248793...
2,None,LC3,(LINESTRING (476084.3086366151 6992433.1708099...
3,None,LC4,(LINESTRING (477864.7330391073 6993715.7889453...
4,None,NPR1,(LINESTRING (477400.3282859836 6996078.3893649...


In [15]:
def get_ends(line, reach,ptlist,reachlist,lenlist):
    if type(line) is MultiLineString:
        line = line[0]
    end_points = line_ends(line)
    ptlist += end_points
    reachlist += [reach,reach]
    lenlist += [line.length, line.length]
    


In [16]:
ptlist = []
reachlist = []
lenlist = []
applyer = lambda x: get_ends(x['geometry'], x['Reach'], ptlist = ptlist, reachlist = reachlist, lenlist = lenlist)
thalwegs.apply(applyer, axis=1)

out = {'geometry':ptlist, 'Reach':reachlist, 'linelength':lenlist}

In [17]:
for cfg_test, cat_name in zip([cfg.pine_dirs, cfg.bris_dirs, cfg.kobble_dirs, cfg.mary_dirs], ['NPR', 'Bris','kobble', 'Mary']):
    os.chdir(cfg_test['folder'])
    thalwegs = gpd.read_file(cfg_test['thalwegs'])
    if 'Reach' not in list(thalwegs.columns):
        thalwegs = thalwegs.rename(columns={'reach':'Reach'})
    
    ptlist = []
    reachlist = []
    lenlist = []
    applyer = lambda x: get_ends(x['geometry'], x['Reach'], ptlist = ptlist, reachlist = reachlist, lenlist = lenlist)
    thalwegs.apply(applyer, axis=1)
    
    out = {'geometry':ptlist, 'Reach':reachlist, 'linelength':lenlist}
    gdf_out = gpd.GeoDataFrame(out, geometry = 'geometry')
    gdf_out.crs = thalwegs.crs
    
    os.chdir(r'E:\Brisbane_Pine GIS QGIS_2018\GIS_scripts_dendro')
    gdf_out.to_file(cat_name+'_end_pts.geojson', driver='GeoJSON')

In [9]:
thalwegs

,id,reach,geometry
0,0.0,MR1,(LINESTRING (471502.7699508757 7043643.6756773...
1,1.0,MR2,(LINESTRING (473039.8303955022 7057845.3384054...
2,2.0,MR3,(LINESTRING (473379.1643790967 7058273.8623458...
3,3.0,MR4,(LINESTRING (473298.658448137 7059506.55797679...
4,NaN,MR1B,(LINESTRING (468637.1523883796 7049798.8285008...
